In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical

In [3]:
# Paths to your dataset folders
spitting_path = "dataset/spitting"
not_spitting_path = "dataset/notspitting"

IMG_SIZE = 64   # Resize all frames
data = []
labels = []

def extract_frames(video_path, label):
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_count += 1
        if frame_count % 5 == 0:   # Take every 5th frame
            frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
            data.append(frame)
            labels.append(label)
    cap.release()

# Extract frames from both folders
for video in os.listdir(spitting_path):
    extract_frames(os.path.join(spitting_path, video), 1)

for video in os.listdir(not_spitting_path):
    extract_frames(os.path.join(not_spitting_path, video), 0)

data = np.array(data) / 255.0
labels = np.array(labels)
print("Dataset shape:", data.shape, labels.shape)


Dataset shape: (2926, 64, 64, 3) (2926,)


In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    data, labels, test_size=0.2, random_state=42
)

y_train = to_categorical(y_train, 2)
y_test = to_categorical(y_test, 2)

print("Train:", X_train.shape, "Test:", X_test.shape)


Train: (2340, 64, 64, 3) Test: (586, 64, 64, 3)


In [5]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D((2,2)),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 31, 31, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 14, 14, 64)        0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 12544)             0         
                                                                 
 dense (Dense)               (None, 128)               1

In [6]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=10,
    batch_size=32
)

model.save("spitting_model.h5")


Epoch 1/10
74/74 [==============================] - 13s 108ms/step - loss: 0.2378 - accuracy: 0.8966 - val_loss: 0.0269 - val_accuracy: 0.9915
Epoch 2/10
74/74 [==============================] - 8s 105ms/step - loss: 0.0232 - accuracy: 0.9940 - val_loss: 0.0108 - val_accuracy: 0.9983
Epoch 3/10
74/74 [==============================] - 8s 104ms/step - loss: 0.0167 - accuracy: 0.9962 - val_loss: 0.0026 - val_accuracy: 1.0000
Epoch 4/10
74/74 [==============================] - 8s 104ms/step - loss: 0.0042 - accuracy: 0.9991 - val_loss: 2.9067e-04 - val_accuracy: 1.0000
Epoch 5/10
74/74 [==============================] - 8s 105ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 2.8204e-04 - val_accuracy: 1.0000
Epoch 6/10
74/74 [==============================] - 8s 106ms/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 4.6737e-04 - val_accuracy: 1.0000
Epoch 7/10
74/74 [==============================] - 8s 103ms/step - loss: 4.9304e-04 - accuracy: 1.0000 - val_loss: 1.3514e-04 - val_ac

d:\KAWIN T K\PrOgRaMmInG\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [3]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load your trained model
model = load_model("spitting_model.h5")

# Define the input image size expected by the model
IMG_SIZE = 64  # change to whatever size your model uses

# Open the webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess the frame
    img = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
    img = img.astype("float32") / 255.0
    img = np.expand_dims(img, axis=0)

    # Prediction
    pred = model.predict(img)[0]

    # If your model uses softmax (two classes):
    label = np.argmax(pred)  

    # If your model uses sigmoid (binary output):
    # label = 1 if pred > 0.5 else 0

    if label == 1:
        text = "Spitting"
        color = (0, 0, 255)  # Red
    else:
        text = "Not Spitting"
        color = (0, 255, 0)  # Green

    # Display label on the frame
    cv2.putText(frame, text, (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
    cv2.imshow("Spitting Detection", frame)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 20ms/step
